# Differences between CEMS-reporting and non-CEMS-reporting plants

Some key characteristics (from Greg):
* Nameplate capacity
* Capacity factor
* Primary fuel type
* Heat rate
* Prime mover type, esp for natural gas plants

In [4]:
import pandas as pd
import scipy.stats as stats
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
%reload_ext autoreload
%autoreload 2

# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

from src.load_data import load_pudl_table
from src.sample_plants import rv_histogramdd

/Users/gailin.pease/miniconda3/envs/hourly_egrid/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [6]:
year = 2020

In [7]:
# gather plant IDs of reporting plants from cems data

cems = pd.read_csv(f"../data/output/cems_{year}_cleaned_20220415.csv")

In [8]:
# load 890 data, 923 data

eia890 = load_pudl_table("generators_eia860", year=year)
eia923 = load_pudl_table("generation_fuel_eia923", year=year)
gen923 = load_pudl_table("generation_eia923", year=year)

In [9]:
all_ids = set(eia890.plant_id_eia.unique())
all_ids.update(set(eia923.plant_id_eia.unique()))
all_ids.update(set(cems.plant_id_eia.unique()))

In [10]:
plants = pd.DataFrame(index = all_ids)
plants["in_CEMS"] = False
plants.loc[cems.plant_id_eia.unique(),"in_CEMS"] = True

In [11]:
px.pie(plants, names="in_CEMS")

# Capacity

In [12]:
plants["capacity"] = eia890.groupby("plant_id_eia").sum().capacity_mw

In [13]:
fig = px.histogram(plants, x="capacity", color="in_CEMS", log_y=True)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

# Generation, capacity factor

In [14]:
plants["generation"] = eia923.groupby("plant_id_eia").sum().net_generation_mwh

In [15]:
fig = px.histogram(plants, x="generation", color="in_CEMS", log_y=True)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [16]:
# todo I'm sure there's a built in python function for this
if year%4 == 0: 
    n_hours = 366*24
else:
    n_hours = 365*24

plants["capacity_factor"] = (plants["generation"]/n_hours)/plants["capacity"]

In [17]:
fig = px.histogram(plants, x="capacity_factor", color="in_CEMS")
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [18]:
plants

,in_CEMS,capacity,generation,capacity_factor
1,False,4.0,3.470000e+02,0.009876
2,False,53.9,1.391700e+05,0.293944
3,True,3615.5,1.049915e+07,0.330593
4,False,225.0,5.546130e+05,0.280618
7,False,138.0,5.043500e+04,0.041606
...,...,...,...,...
64876,False,10.0,NaN,NaN
64877,False,36.0,NaN,NaN
64878,False,202.9,NaN,NaN
64879,False,184.0,NaN,NaN


# Heat rate

Heat rate = energy consumed / generation, in mmBtu/MWh

In [19]:
plants["fuel_consumed"] = eia923.groupby("plant_id_eia").sum().fuel_consumed_mmbtu
plants["heat_rate"] = plants["fuel_consumed"]/plants["generation"]
# assume heat_rate = 0 should be NaN, these are plants that didn't consume anything
plants.loc[plants["heat_rate"]==0,"heat_rate"] = np.nan

In [20]:
fig = px.histogram(plants, x="heat_rate", color="in_CEMS", log_y=True)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

# Use eGRID 

This avoids potential data issues with 860, 923 that are fixed by eGRID

In [126]:
egrid_plant = pd.read_excel(f'../data/egrid/egrid{year}_data.xlsx', 
                            sheet_name=f'PLNT{str(year)[-2:]}', 
                            header=1, 
                            usecols=["ORISPL", # Plant code
                            "BACODE", # BA code
                            "PLHTRT", # heat rate
                             "CAPFAC",# capacity factor
                             "NAMEPCAP",# nameplate capacity
                             "CHPFLAG", # combined heat and power
                             "ELCALLOC", # CHP electric allocation factor
                             "PLCO2AN", # annual CO2 emissions (tons)
                             "PLPRMFL", # plant primary fuel
                             "PLFUELCT", # plant fuel category 
                             "NUMUNT", # number of units
                             "NUMGEN", # number of generators
                             "PLNGENAN"]) # annual generation 


In [127]:
# Fix eGRID IDs
# TODO move into helper function, this code is reused between here and data_pipeline

egrid_crosswalk = pd.read_csv('../data/egrid/egrid_static_tables/2020/table_C5_crosswalk_of_EIA_ID_to_EPA_ID.csv')
epaid_to_eiaid = dict(zip(list(egrid_crosswalk['plant_id_egrid']), list(egrid_crosswalk['plant_id_eia'])))
print(f" Updatating {len(egrid_plant[egrid_plant['ORISPL'].isin(list(egrid_crosswalk['plant_id_egrid']))])} plant codes from eGRID")

egrid_plant['plant_id_eia'] = egrid_plant['ORISPL'].map(lambda x: epaid_to_eiaid.get(x, x))
egrid_plant = egrid_plant.set_index("plant_id_eia")

 Updatating 29 plant codes from eGRID


In [128]:
# filter for fossil plants
fossil = egrid_plant["PLFUELCT"].isin(["COAL","OIL","GAS","OFSL","BIOMASS","OTHF"])
print(f"{sum(fossil)} fossil plants in {len(egrid_plant)} egrid plants")
egrid_plant = egrid_plant[fossil]

4508 fossil plants in 12668 egrid plants


In [129]:
cems_reporters = cems.plant_id_eia.unique()
print(f"{len(set(cems_reporters).difference(egrid_plant.index.unique()))} CEMS plants not in eGRID")
cems_reporters = list(set(cems_reporters).intersection(egrid_plant.index.unique()))

egrid_plant["in_CEMS"] = False
egrid_plant.loc[cems_reporters, "in_CEMS"] = True

20 CEMS plants not in eGRID


In [130]:
print(f"{sum(egrid_plant.in_CEMS)} plants in CEMS out of {len(egrid_plant)}")

1242 plants in CEMS out of 4508


In [140]:
px.pie(egrid_plant, names="in_CEMS")

In [132]:
fig = px.histogram(egrid_plant, x="NAMEPCAP", color="in_CEMS", log_y=False, title="Capacity", histnorm='probability',
facet_col="PLFUELCT", facet_col_wrap=2)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [134]:
fig = px.histogram(egrid_plant, x="CAPFAC", color="in_CEMS", log_y=False, title="Capacity factor", histnorm='probability',
facet_col="PLFUELCT", facet_col_wrap=2)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [135]:
fig = px.histogram(egrid_plant, x="PLHTRT", color="in_CEMS", log_y=False, title="Heat rate", histnorm='probability', facet_col="PLFUELCT", facet_col_wrap=2)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [87]:
def make_variable_bin_bounds(start, start_high_res, end_high_res, end, n_high_res=10, n_total=10):
    """
    Entire span (start to stop) is binned with n_total bins. 
    Between start_high_res and end_high_res, bins are replaced with n_high_res bins
    for a total of n_total + n_high_res + 2 bins (max)
    """
    wide_bins = np.linspace(start, end, num=n_total+1)
    # discard wide bin boundaries between high res bins
    wide_bins = wide_bins[(wide_bins<start_high_res) | (wide_bins>end_high_res)]
    narrow_bins = np.linspace(start_high_res, end_high_res, num=n_high_res+1)
    # Insert narrow bin boundaries 
    ii = np.searchsorted(wide_bins, narrow_bins)
    return np.insert(wide_bins, ii, narrow_bins)

In [93]:
fig = px.histogram(egrid_plant, x="PLCO2AN", color="in_CEMS", log_y=False, title="Annual CO2 emissions", histnorm='probability')
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [90]:
fig = px.histogram(egrid_plant, x="PLNGENAN", color="in_CEMS", log_y=False, title="Annual generation", histnorm='probability')
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

# Emperical rejection sampling of CEMS plants 

Goal: get plants with distribution of properties similar to non-CEMS plants.

* np.histogramdd: multidimensional histogram, preferred since this would allow vars to be dependent (eg, capacity and capacity factor) 
* np.histogram: single dimensional histogram, not preferred since its use would imply independence between vars
* scipy.stats.rv_histogram https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_histogram.html 

In [141]:
cems_plants = egrid_plant[egrid_plant.in_CEMS == True]
non_cems_plants = egrid_plant[egrid_plant.in_CEMS == False]

In [104]:
numeric_cols = ["PLHTRT", # heat rate
             "CAPFAC",# capacity factor
             "NAMEPCAP",# nameplate capacity
            #  "PLCO2AN", # annual CO2 emissions (tons)
             "PLNGENAN", # annual generation
             ] 



In [116]:
cap_fact_bins = make_variable_bin_bounds(0,0,.2,1)
htrt_bins = make_variable_bin_bounds(-500000, -10000, 40000, 500000)
cap_bins = make_variable_bin_bounds(0, 0, 500, 6000, n_high_res=15)
gen_bins = make_variable_bin_bounds(0, 0, 600000, 15000000)

bins = [htrt_bins, cap_fact_bins, cap_bins, gen_bins]


In [117]:
to_fit_cems = cems_plants[numeric_cols].dropna(how="any").to_numpy()
histdd_cems = np.histogramdd(to_fit_cems, bins=bins, density=True)
rv_cems = rv_histogramdd(histdd_cems)

to_fit_nocem = non_cems_plants[numeric_cols].dropna(how="any").to_numpy()
histdd_nocem = np.histogramdd(to_fit_nocem, bins=bins, density=True)
rv_no_cems = rv_histogramdd(histdd_nocem)

In [118]:
rv_no_cems.nbins

[20, 18, 25, 20]

In [119]:
px.density_heatmap(non_cems_plants, x=numeric_cols[1], y=numeric_cols[0], marginal_x="histogram", marginal_y="histogram")

In [120]:
# Create column to flag selected plants. Numeric because some plants may be selected multiple times 
egrid_plant["selected"] = 0
# Rejection sampling: 
#  1) sample from CEMS plants 
#  2) calculate probability under target dist (rv_no_cems) and source dist (rv_cems)
#  3) accept with probability M*P_target(x)/P_source(x) where M is scaling factor to keep P<1
N = 1000
M = 1600 # scaling factor chosen emperically 
selected = 0 
while selected < N:
    plant = cems_plants.sample()
    x = plant[numeric_cols].to_numpy()[0,:]
    p_source = rv_cems.pdf(x)
    if p_source == 0: 
        continue
    p_accept = (1/M)*rv_no_cems.pdf(x)/p_source
    if np.random.random() > p_accept:
        egrid_plant.loc[plant.index,"selected"] += 1
        selected += 1
    if p_accept > 1:
        print(f"Warning: acceptance prob {p_accept}")


In [121]:
# Plot bins against data histograms to visualize emperical dist 
vars = rv_no_cems.nvars

fig = make_subplots(5,1,subplot_titles=numeric_cols,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}]])
for i in range(vars):
    tosum = list(range(vars))
    tosum.remove(i)
    fig.add_trace(go.Histogram(x=cems_plants[numeric_cols[i]], opacity=.75, marker_color="lightblue", name="CEMS plants", histnorm='probability'),
         i+1, 1, secondary_y=True)
    fig.add_trace(go.Histogram(x=non_cems_plants[numeric_cols[i]], opacity=.75, marker_color="pink", name="Non-CEMS plants", histnorm="probability"),
         i+1, 1, secondary_y=True)

    bin_centers = (rv_cems._hbins[i][1:] + rv_cems._hbins[i][:-1])/2
    fig.add_trace(go.Scatter(x=bin_centers, y=rv_cems._hpdf.sum(axis=tuple(tosum)), marker_color="blue", name="CEMS fit"), i+1, 1, secondary_y=False)
    
    bin_centers = (rv_no_cems._hbins[i][1:] + rv_no_cems._hbins[i][:-1])/2
    fig.add_trace(go.Scatter(x=bin_centers, y=rv_no_cems._hpdf.sum(axis=tuple(tosum)), marker_color="red", name="Non-CEMS fit"), i+1, 1, secondary_y=False)

fig.update_layout(height=600, width=600, barmode='overlay')
fig.show()

In [122]:
fig = make_subplots(5,1, subplot_titles=numeric_cols)

for i in range(vars):
    tosum = list(range(vars))
    tosum.remove(i)

    # Some selected rows are repeated: make new DF reflecting that
    selected = egrid_plant.loc[egrid_plant.index.repeat(egrid_plant['selected'])]
    fig.add_trace(go.Histogram(x=selected.loc[:,numeric_cols[i]], 
        opacity=.5, histnorm='probability', name="Sampled", marker_color='blue'), i+1, 1)
    fig.add_trace(go.Histogram(x=egrid_plant.loc[egrid_plant.in_CEMS==False,numeric_cols[i]], 
        opacity=.5, histnorm='probability', name="Not in CEMS", marker_color='red'), i+1, 1)
    fig.add_trace(go.Histogram(x=egrid_plant.loc[egrid_plant.in_CEMS==True,numeric_cols[i]], 
        opacity=.5, histnorm='probability', name="In CEMS", marker_color='green'), i+1, 1)

fig.update_layout(height=600, width=600)
fig.update_layout(barmode='overlay')

fig.show()

In [123]:
egrid_plant.loc[egrid_plant.index.repeat(egrid_plant['selected'])].head(10)

,ORISPL,NUMUNT,NUMGEN,PLPRMFL,PLFUELCT,CAPFAC,NAMEPCAP,CHPFLAG,ELCALLOC,PLNGENAN,PLCO2AN,PLHTRT,in_CEMS,selected
plant_id_eia,,,,,,,,,,,,,,
3,3,8,13,BIT,COAL,0.33150,3615.5,NaN,NaN,1.049915e+07,6846726.509,8752.700178,True,1
55409,55409,4,4,NG,GAS,0.00598,748.0,NaN,NaN,3.921100e+04,25547.399,10779.405163,True,1
55440,55440,3,4,NG,GAS,0.62212,927.1,NaN,NaN,5.052482e+06,2391941.250,7966.166332,True,2
55440,55440,3,4,NG,GAS,0.62212,927.1,NaN,NaN,5.052482e+06,2391941.250,7966.166332,True,2
47,47,8,13,NG,GAS,0.00018,1826.0,NaN,NaN,2.952000e+03,3304.100,18097.899894,True,2
47,47,8,13,NG,GAS,0.00018,1826.0,NaN,NaN,2.952000e+03,3304.100,18097.899894,True,2
55292,55292,3,4,NG,GAS,0.18751,902.4,Yes,1.000000,1.482286e+06,654223.000,7426.729727,True,1
7897,7897,4,8,NG,GAS,0.19904,2534.0,NaN,NaN,4.418244e+06,1828239.969,6962.875070,True,1
10,10,11,11,NG,GAS,0.09491,1288.4,NaN,NaN,1.071142e+06,696954.494,10868.539908,True,1


# This is not a good strategy 

Instead, pick smallest capacity plant per BA and fuel type.

In [161]:
ids = cems_plants.groupby(["BACODE","PLFUELCT"]).agg(
    plant_id_eia=pd.NamedAgg(column="NAMEPCAP", aggfunc='idxmin'))

In [171]:
egrid_plant["plants_to_compare"] = False
egrid_plant.loc[ids['plant_id_eia'], "plants_to_compare"] = True


In [176]:
fig = px.histogram(egrid_plant, x="NAMEPCAP", color="in_CEMS", log_y=False, title="Capacity", histnorm='probability')

# Add comparison plants
fig.add_trace(go.Histogram(x=egrid_plant.loc[egrid_plant.plants_to_compare == True, "NAMEPCAP"], histnorm='probability', nbinsx=200))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)


In [177]:
egrid_plant.head()

,ORISPL,BACODE,NUMUNT,NUMGEN,PLPRMFL,PLFUELCT,CAPFAC,NAMEPCAP,CHPFLAG,ELCALLOC,PLNGENAN,PLCO2AN,PLHTRT,in_CEMS,plants_to_compare
plant_id_eia,,,,,,,,,,,,,,,
54452,54452,NaN,12,12,NG,GAS,NaN,21.6,Yes,NaN,NaN,NaN,NaN,False,False
57053,57053,NaN,5,5,DFO,OIL,0.00000,2.6,Yes,1.0,0.000,NaN,NaN,False,False
60243,60243,NaN,4,4,DFO,OIL,0.09224,1.5,NaN,NaN,1212.000,943.616,9533.827577,False,False
75,75,CEA,8,8,NG,GAS,0.00719,121.4,NaN,NaN,7646.000,6724.603,15043.813248,False,False
7462,7462,NaN,4,4,DFO,OIL,0.09865,1.9,Yes,1.0,1641.999,1348.008,10052.983571,False,False


In [179]:
fig = px.histogram(egrid_plant, x="PLHTRT", color="in_CEMS", log_y=False, title="Heat rate", histnorm='probability')

# Add comparison plants
fig.add_trace(go.Histogram(x=egrid_plant.loc[egrid_plant.plants_to_compare == True, "PLHTRT"], histnorm='probability', nbinsx=200))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [183]:
fig = px.histogram(egrid_plant, x="CAPFAC", color="in_CEMS", log_y=False, title="Capacity factor", histnorm='probability')

# Add comparison plants
fig.add_trace(go.Histogram(x=egrid_plant.loc[egrid_plant.plants_to_compare == True, "CAPFAC"], histnorm='probability', nbinsx=75))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

In [185]:
plants_to_compare = egrid_plant[egrid_plant.plants_to_compare==True]
plants_to_compare.to_csv("../data/output/validation_plants.csv")